# Constellation Orbit Computation

This tutorial demonstrates how to generate satellite orbits for a constellation with pattern Walker-Delta

## Setup

In [ ]:
import numpy as np
import pandas as pd

import plotly.graph_objs as go

from ostk.physics.units import Length
from ostk.physics.units import Angle
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth

from ostk.astrodynamics import Trajectory
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory.orbit.models import Kepler
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE
from ostk.astrodynamics.trajectory.orbit.models import SGP4
from ostk.astrodynamics.trajectory.orbit.models.sgp4 import TLE

---

### Define Simulation Characteristics

In [ ]:
epoch = Instant.date_time(DateTime(2022, 1, 1, 0, 0, 0), Scale.UTC)

In [ ]:
environment = Environment.default()
earth = environment.access_celestial_object_with_name('Earth')

### Define Constellation Characteristics

In [ ]:
number_of_satellites = 24
number_of_planes = 4
semi_major_axis = Length.kilometers(7000.0)
inclination = Angle.degrees(70.0)
eccentricity = 0.00001
phasing = 0.0

In [ ]:
def get_walker_delta_orbits (number_of_satellites, 
                             number_of_planes,
                             epoch,
                             semi_major_axis,
                             inclination,
                             eccentricity,
                             phasing):
    
    '''
    Returns the orbits of the different satellites for a Walker-Delta constellation pattern.
    '''
    
    orbits = {}
    
    # Get number of Satellites per plane
    number_sat_per_plane = number_of_satellites // number_of_planes
    
    # Define perturbation type in Kepler model
    perturbation_type = Kepler.PerturbationType.J4
    
    # Assume equally spaced planes and equally phased satellites in each plane (Walker constellation)
    raan = 0.0
    aop = 0.0
    true_anomaly = 0.0
    satellite_counter = 1
    
    for i in range(number_of_planes):
        
        for j in range(number_sat_per_plane):
            
            coe = COE(
                semi_major_axis,
                eccentricity,
                inclination,
                Angle.degrees(raan),
                Angle.degrees(aop),
                Angle.degrees(true_anomaly)
            )
            
            kepler = Kepler(
                coe,
                epoch,
                earth,
                perturbation_type
            )

            orbits[satellite_counter] = Orbit(kepler, earth)

            true_anomaly += 360.0 / number_sat_per_plane
            satellite_counter += 1

        true_anomaly += phasing * 360 / number_of_satellites
        raan += 360 / number_of_planes
        
    return orbits

In [ ]:
orbits = get_walker_delta_orbits (
    number_of_satellites, 
    number_of_planes,
    epoch,
    semi_major_axis,
    inclination,
    eccentricity,
    phasing
)

### Propagate Orbits

In [ ]:
start_instant = Instant.date_time(DateTime(2022, 1, 1, 0, 0, 0), Scale.UTC)
end_instant = Instant.date_time(DateTime(2022, 1, 1, 0, 30, 0), Scale.UTC)

In [ ]:
interval = Interval.closed(start_instant, end_instant)

In [ ]:
step = Duration.minutes(1.0)

In [ ]:
instants = interval.generate_grid(step)

In [ ]:
def convert_state (instant, state):
    
    lla = LLA.cartesian(
        state.get_position().in_frame(Frame.ITRF(), state.get_instant()).get_coordinates(), 
        Earth.equatorial_radius, 
        Earth.flattening
    )
    
    return [
                repr(instant),
                float(instant.get_modified_julian_date(Scale.UTC)),
                *state.get_position().get_coordinates().tolist(),
                *state.get_velocity().get_coordinates().tolist(),
                float(lla.get_latitude().in_degrees()),
                float(lla.get_longitude().in_degrees()),
                float(lla.get_altitude().in_meters())
            ]

In [ ]:
def propagate_kepler_orbit (orbit, instants):
    
    states = [[instant, orbit.get_state_at(instant)] for instant in instants]
    orbit_data = [convert_state(instant, state) for [instant, state] in states]
    
    orbit_df = pd.DataFrame(
        orbit_data, 
        columns = [
            '$Time^{UTC}$', 
            '$MJD^{UTC}$', 
            '$x_{x}^{ECI}$', 
            '$x_{y}^{ECI}$', 
            '$x_{z}^{ECI}$', 
            '$v_{x}^{ECI}$', 
            '$v_{y}^{ECI}$', 
            '$v_{z}^{ECI}$', 
            '$Latitude$', 
            '$Longitude$', 
            '$Altitude$'
        ]
    )
    
    return orbit_df

In [ ]:
orbit_dfs = [propagate_kepler_orbit(orbit[1], instants) for orbit in orbits.items()]

### Output

2D plot, over **World Map**:

In [ ]:
data = []

for satellite_index in range(number_of_satellites):
    
    orbit_df = orbit_dfs[satellite_index]
    
    data.append(
        go.Scattergeo(
            lon = orbit_df['$Longitude$'],
            lat = orbit_df['$Latitude$'],
            mode = 'lines',
            name = f'Satellite {satellite_index}',
            line = go.scattergeo.Line(
                width = 1
            )
        )
    )
    
figure = go.Figure(
    data = data,
    layout = go.Layout(
        title = None,
        showlegend = False,
        height=1000,
        geo = go.layout.Geo(
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)'
        )
    )
)

figure.show()

3D plot, in **Earth Fixed** frame:

In [ ]:
data = []

for satellite_index in range(number_of_satellites):
    
    orbit_df = orbit_dfs[satellite_index]
    
    data.append(
        go.Scattergeo(
            lon = orbit_df['$Longitude$'],
            lat = orbit_df['$Latitude$'],
            mode = 'lines',
            name = f'Satellite {satellite_index}',
            line = go.scattergeo.Line(
                width = 2,
            )
        )
    )

figure = go.Figure(
    data = data,
    layout = go.Layout(
        title = None,
        showlegend = False,
        width = 800,
        height = 800,
        geo = go.layout.Geo(
            showland = True,
            showlakes = True,
            showcountries = False,
            showocean = True,
            countrywidth = 0.0,
            landcolor = 'rgb(100, 100, 100)',
            lakecolor = 'rgb(240, 240, 240)',
            oceancolor = 'rgb(240, 240, 240)',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),
            lonaxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            ),
            lataxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            )
        )
    )
)

figure.show()

3D plot, in **Earth Inertial** frame:

In [ ]:
data = []

theta = np.linspace(0, 2 * np.pi, 30)
phi = np.linspace(0, np.pi, 30)

theta_grid, phi_grid = np.meshgrid(theta, phi)

r = float(Earth.equatorial_radius.in_meters())

x = r * np.cos(theta_grid) * np.sin(phi_grid)
y = r * np.sin(theta_grid) * np.sin(phi_grid)
z = r * np.cos(phi_grid)

earth = go.Surface(
    x = x,
    y = y,
    z = z,
    colorscale = 'Viridis',
    showscale = False
)

data.append(earth)

for satellite_index in range(number_of_satellites):
    
    orbit_df = orbit_dfs[satellite_index]

    trace = go.Scatter3d(
        x = orbit_df['$x_{x}^{ECI}$'],
        y = orbit_df['$x_{y}^{ECI}$'],
        z = orbit_df['$x_{z}^{ECI}$'],
        mode = 'lines',
        name = f'Satellite {satellite_index}',
        marker = dict(
            size = 0,
#             color = initial_orbit_df['$x_{z}^{ECI}$'],
            colorscale = 'Viridis',
            showscale = False
        ),
        line = dict(
#             color = initial_orbit_df['$x_{z}^{ECI}$'],
            width = 3
        )
    )
    
    data.append(trace)

figure = go.Figure(
    data = data,
    layout = go.Layout(
        title = None,
        width = 800,
        height = 1000,
        showlegend = False,
        scene = go.layout.Scene(
            xaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                backgroundcolor = 'rgb(230, 230,230)'
            ),
            yaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                backgroundcolor = 'rgb(230, 230,230)'
            ),
            zaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                backgroundcolor = 'rgb(230, 230,230)'
            ),
            camera = dict(
                up = dict(
                    x = 0,
                    y = 0,
                    z = 1
                ),
                eye = dict(
                    x = -1.7428,
                    y = 1.0707,
                    z = 0.7100,
                )
            ),
        )
    )
)

figure.show()

---